In [39]:
import pandas as pd
import numpy as np
import plotly.express as px
from IPython.display import display
import datetime as dt

from genres import categorize_genres

In [40]:
df_film_wiki = pd.read_csv('film_2015_2024.csv')
df_film_cmu = pd.read_csv('movie_cmu.csv')

display(df_film_wiki.head(1))
display(df_film_cmu.head(1))

,page_id,wikidata_id,film,release_date,box_office,runtime,languages,countries,genres,reviewScores,awardsReceived,awardsNominated,capitalCost,release date
0,48716179,Q21646407,First Rank Raju,2015-01-01T00:00:00Z,NaN,NaN,Kannada,India,['drama'],[''],[''],[''],NaN,2015


,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."


First we start by renaming the columns so that they match between the two datasets

In [41]:
df_film_cmu = df_film_cmu.rename(columns={'Wikipedia movie ID':'wikipedia_id', 'Freebase movie ID': 'dataset_id', 'Movie name': 'title', 'Movie release date': 'release_date', 'Movie box office revenue': 'box_office', 'Movie runtime': 'runtime', 'Movie languages': 'languages', 'Movie countries': 'countries', 'Movie genres': 'genres'})

In [42]:
df_film_wiki = df_film_wiki.rename(columns={'page_id':'wikipedia_id', 'wikidata_id': 'dataset_id', 'film': 'title'})

In [43]:
display(df_film_wiki.head())
display(df_film_cmu.head())

,wikipedia_id,dataset_id,title,release_date,box_office,runtime,languages,countries,genres,reviewScores,awardsReceived,awardsNominated,capitalCost,release date
0,48716179,Q21646407,First Rank Raju,2015-01-01T00:00:00Z,NaN,NaN,Kannada,India,['drama'],[''],[''],[''],NaN,2015
1,46967307,Q20311376,A 2nd Hand Lover,2015-01-01T00:00:00Z,NaN,NaN,Kannada,India,['musical'],[''],[''],[''],NaN,2015
2,48837152,Q24521064,"3 1/2 Minutes, 10 Bullets",2015-01-24T00:00:00Z,NaN,98,English,United States of America,"['documentary', 'crime']","['100%', '7.7/10']",['Sundance Special Jury Prize Documentary'],[''],NaN,2015
3,44838624,Q18701869,3 Bahadur,2015-01-01T00:00:00Z,NaN,94,Urdu,Pakistan,"['adventure', ""children's""]",[''],[''],[''],NaN,2015
4,46896101,Q20770561,3 Generations,2015-09-18T00:00:00Z,NaN,87,"English, Spanish",United States of America,"['drama', 'LGBT-related', 'comedy']","['31%', '5/10']",[''],[''],NaN,2015


,wikipedia_id,dataset_id,title,release_date,box_office,runtime,languages,countries,genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


Now we will parse the date columns so that it's in the same format for both datasets

In [44]:
def parse_date_wiki(date):
    try:
        if date == 'NaN':
            return np.nan
        return dt.datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ").date()
    except:
        return np.nan
    
    
df_film_wiki['release_date'] = df_film_wiki['release_date'].apply(parse_date_wiki)

In [45]:
def parse_date_cmu(date):
    try:
        if date.isdigit():
            return dt.datetime.strptime(date, "%Y").date()
        elif date.count('-') == 1:
            return dt.datetime.strptime(date, "%Y-%m").date()
        elif date.count('-') == 2:
            return dt.datetime.strptime(date, "%Y-%m-%d").date()
        else:
            return np.nan
    except:
        return np.nan
    
df_film_cmu['release_date'] = df_film_cmu['release_date'].apply(parse_date_cmu)

Now we will cast all the runtimes to integers

In [46]:
def parse_runtime(x):
    try: 
        if x == 'NaN':
            return np.nan
        return int(x)
    except:
        return np.nan


df_film_cmu['runtime'] = df_film_cmu['runtime'].apply(parse_runtime)
df_film_wiki['runtime'] = df_film_wiki['runtime'].apply(parse_runtime)

Now we will normalize the language column so that it's in the same format for both datasets

In [47]:
def parse_language_wiki(languages):
    try:
        languages = languages.split(',')
        languages = [language.strip() for language in languages]
        return languages
    except:
        return np.nan
    
df_film_wiki['languages'] = df_film_wiki['languages'].apply(parse_language_wiki)

In [48]:
from operator import contains


def parse_language_cmu(languages):
    try:
        languages = languages.replace('{','').replace('}','').replace('"','')
        languages = languages.split(',')
        languages = [language.split(':') for language in languages]
        languages = [language[1].strip() for language in languages]
        languages = [language.split(' ')[0] if contains(language.lower(), 'language')  else language for language in languages]
        return languages
    except:
        return np.nan
    
df_film_cmu['languages'] = df_film_cmu['languages'].apply(parse_language_cmu)

Now lets normalize the countries column so that it's in the same format for both datasets

In [49]:
def parse_country_wiki(countries):
    try:
        countries = countries.split(',')
        countries = [country.strip() for country in countries]
        return countries
    except:
        return np.nan
    
df_film_wiki['countries'] = df_film_wiki['countries'].apply(parse_country_wiki)

In [50]:
def parse_country_cmu(countries):
    try:
        countries = countries.replace('{','').replace('}','').replace('"','')
        countries = countries.split(',')
        countries = [country.split(':') for country in countries]
        countries = [country[1].strip() for country in countries]
        return countries
    except:
        return np.nan
    
df_film_cmu['countries'] = df_film_cmu['countries'].apply(parse_country_cmu)

Now we will normalize the genres column so that it's in the same format for both datasets

In [51]:
def parse_genre_wiki(genres):
    try:
        genres = genres.replace('[','').replace(']','').replace("'",'').replace('"','')
        genres = genres.split(',')
        genres = [genre.strip() for genre in genres]
        return genres
    except:
        return None
    
df_film_wiki['genres'] = df_film_wiki['genres'].apply(parse_genre_wiki)

In [52]:
def parse_genre_cmu(genres):
    try:
        genres = genres.replace('{','').replace('}','').replace('"','')
        genres = genres.split(',')
        genres = [genre.split(':') for genre in genres]
        genres = [genre[1].strip() for genre in genres]
        return genres
    except:
        return None
    
df_film_cmu['genres'] = df_film_cmu['genres'].apply(parse_genre_cmu)

df_film_cmu

,wikipedia_id,dataset_id,title,release_date,box_office,runtime,languages,countries,genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,[English],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,[English],[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]"
2,28463795,/m/0crgdbh,Brun bitter,1988-01-01,NaN,83.0,[Norwegian],[Norway],"[Crime Fiction, Drama]"
3,9363483,/m/0285_cd,White Of The Eye,1987-01-01,NaN,110.0,[English],[United Kingdom],"[Thriller, Erotic thriller, Psychological thri..."
4,261236,/m/01mrr1,A Woman in Flames,1983-01-01,NaN,106.0,[German],[Germany],[Drama]
...,...,...,...,...,...,...,...,...,...
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,[English],[United States of America],[Drama]
81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,[English],"[Ireland, United Kingdom]","[Biographical film, Drama, Documentary]"
81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,[English],[United States of America],"[Satire, Comedy]"
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,[Japanese],[Japan],"[Science Fiction, Japanese Movies, Adventure, ..."


In [53]:
df_film_cmu['categories'] = df_film_cmu['genres'].apply(categorize_genres)
df_film_wiki['categories'] = df_film_wiki['genres'].apply(categorize_genres)

Now let's transform the rating column to a numerical value

In [54]:
def get_numerical_review(review):
    if contains(review, '%'):
        return int(review.replace('%',''))/100
    elif contains(review, '/'):
        return float(review.split('/')[0])/float(review.split('/')[1])
    else:
        return np.nan

def parse_review(reviews):
    try:
        reviews = reviews.replace('[','').replace(']','').replace("'",'')
        reviews = reviews.split(',')
        reviews = [get_numerical_review(review.strip()) for review in reviews]
        return np.mean(reviews)
    except:
        return np.nan
    
df_film_wiki['reviewScores'] = df_film_wiki['reviewScores'].apply(parse_review)

Now let's parse the awards

In [55]:
def parse_awards(awards):
    try:
        awards = awards.replace('[','').replace(']','').replace("'",'')
        awards = awards.split(',')
        awards = [award.strip() for award in awards]
        return awards
    except:
        return []
    
df_film_wiki['awardsNominated'] = df_film_wiki['awardsNominated'].apply(parse_awards)
df_film_wiki['awardsReceived'] = df_film_wiki['awardsReceived'].apply(parse_awards)

df_film_wiki


,wikipedia_id,dataset_id,title,release_date,box_office,runtime,languages,countries,genres,reviewScores,awardsReceived,awardsNominated,capitalCost,release date,categories
0,48716179,Q21646407,First Rank Raju,2015-01-01,NaN,NaN,[Kannada],[India],[drama],NaN,[],[],NaN,2015,[Drama]
1,46967307,Q20311376,A 2nd Hand Lover,2015-01-01,NaN,NaN,[Kannada],[India],[musical],NaN,[],[],NaN,2015,[Music & Dance]
2,48837152,Q24521064,"3 1/2 Minutes, 10 Bullets",2015-01-24,NaN,98.0,[English],[United States of America],"[documentary, crime]",0.885,[Sundance Special Jury Prize Documentary],[],NaN,2015,"[Thriller & Mystery, Documentary]"
3,44838624,Q18701869,3 Bahadur,2015-01-01,NaN,94.0,[Urdu],[Pakistan],"[adventure, childrens]",NaN,[],[],NaN,2015,"[Action & Adventure, Animated & Family]"
4,46896101,Q20770561,3 Generations,2015-09-18,NaN,87.0,"[English, Spanish]",[United States of America],"[drama, LGBT-related, comedy]",0.405,[],[],NaN,2015,"[Drama, Comedy, Romance]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27002,77719767,Q129815195,Yudhra,NaN,NaN,NaN,NaN,NaN,None,NaN,[],[],NaN,NaN,[]
27003,76212902,Q124694939,Yumi's Cells: The Movie,NaN,NaN,NaN,NaN,NaN,None,NaN,[],[],NaN,NaN,[]
27004,74732328,Q122194265,Yuva,2023-12-22,NaN,NaN,NaN,[India],[],NaN,[],[],NaN,2023,[]
27005,77736917,Q130014181,Z-Mom,NaN,NaN,NaN,NaN,NaN,None,NaN,[],[],NaN,NaN,[]


Now let's merge the two datasets

In [56]:
df_film_cmu['languages'] = df_film_cmu['languages'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)
df_film_cmu['countries'] = df_film_cmu['countries'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)
df_film_cmu['genres'] = df_film_cmu['genres'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)
df_film_cmu['categories'] = df_film_cmu['categories'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)

df_film_wiki['languages'] = df_film_wiki['languages'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)
df_film_wiki['countries'] = df_film_wiki['countries'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)
df_film_wiki['genres'] = df_film_wiki['genres'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)
df_film_wiki['categories'] = df_film_wiki['categories'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)
df_film_wiki['awardsNominated'] = df_film_wiki['awardsNominated'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)
df_film_wiki['awardsReceived'] = df_film_wiki['awardsReceived'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)


df_full = pd.merge(df_film_wiki, df_film_cmu, on=['wikipedia_id', 'dataset_id', 'title', 'release_date', 'box_office', 'runtime', 'languages', 'countries', 'genres', 'categories'], how='outer')


df_full['languages'] = df_full['languages'].apply(lambda x: x.split(',') if isinstance(x, str) else x)
df_full['countries'] = df_full['countries'].apply(lambda x: x.split(',') if isinstance(x, str) else x)
df_full['genres'] = df_full['genres'].apply(lambda x: x.split(',') if isinstance(x, str) else x)
df_full['categories'] = df_full['categories'].apply(lambda x: x.split(',') if isinstance(x, str) else x)
df_full['awardsNominated'] = df_full['awardsNominated'].apply(lambda x: x.split(',') if isinstance(x, str) else [])
df_full['awardsReceived'] = df_full['awardsReceived'].apply(lambda x: x.split(',') if isinstance(x, str) else [])



df_full.to_csv('film_full.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'film_full.csv'